# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [32]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:

weatherdf = pd.read_csv("../output_data/city_weather.csv")
weatherdf.head()
clean_weather = weatherdf.dropna(axis=0, how='any', inplace=False)
clean_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,luoyang,34.6836,112.4536,282.04,99.0,87.0,4.03,CN,1.612484e+09
1,ilhabela,-23.7781,-45.3581,299.75,77.0,29.0,1.61,BR,1.612484e+09
2,aden,12.7794,45.0367,296.54,70.0,0.0,7.35,YE,1.612484e+09
3,praya,14.9215,-23.5087,294.15,73.0,75.0,7.20,CV,1.612484e+09
4,port alfred,-33.5906,26.8910,292.04,88.0,16.0,4.02,ZA,1.612484e+09
...,...,...,...,...,...,...,...,...,...
623,maragheh,37.3917,46.2398,279.15,93.0,90.0,2.06,IR,1.612484e+09
624,baton rouge,30.4508,-91.1545,292.15,68.0,90.0,6.69,US,1.612484e+09
625,port-gentil,-0.7193,8.7815,301.15,89.0,40.0,4.12,GA,1.612484e+09
626,samarai,-10.6167,150.6667,302.12,68.0,30.0,1.09,PG,1.612484e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [15]:
#set lat/lng as locations
locations = clean_weather[["Lat", "Lng"]].astype(float)
# weatherdf['Humidity'].dtypes
humidity = clean_weather["Humidity"]
#add heat layer to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
#narrow data by max temp between 80-90 degreed farenheit
warmdf = clean_weather.loc[((clean_weather["Max Temp"]) >299)]
cooldf = warmdf.loc[((warmdf["Max Temp"]) <305)] 
#between 25 and 30 cloudiness
sunnydf = cooldf.loc[((cooldf["Cloudiness"]) <25)]
perfectweather = sunnydf.loc[((sunnydf["Cloudiness"]) >15)]
perfectweather

,City,Lat,Lng,Country
9,harper,4.3750,-7.7169,LR
94,alvaraes,-3.2208,-64.8042,BR
116,oistins,13.0667,-59.5333,BB
119,banjar,-8.1900,114.9675,ID
171,bambous virieux,-20.3428,57.7575,MU
274,the valley,18.2170,-63.0578,AI
474,bathsheba,13.2167,-59.5167,BB
515,itacare,-14.2775,-38.9967,BR


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
#make new hotel df and add hotel name column
hotel_df = perfectweather[["City", "Lat", "Lng", "Country"]]
hotel_df["Hotel Name"] = ""
hotel_df

c:\users\etrez\anaconda3\envs\pythondata2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Country,Hotel Name
9,harper,4.3750,-7.7169,LR,
94,alvaraes,-3.2208,-64.8042,BR,
116,oistins,13.0667,-59.5333,BB,
119,banjar,-8.1900,114.9675,ID,
171,bambous virieux,-20.3428,57.7575,MU,
274,the valley,18.2170,-63.0578,AI,
474,bathsheba,13.2167,-59.5167,BB,
515,itacare,-14.2775,-38.9967,BR,


In [34]:
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
params = {"key": g_key}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_search = "hotel"
radius = 5000
for index, row in hotel_df.iterrows():

    lat = row['Lat']
    lng = row['Lng']
    params = {
        "location": f"{lat},{lng}",
        "types": target_search,
        "radius": radius,
        "key": g_key}
    response = requests.get(base_url, params=params)
    response = response.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except:
        print("Error/missing info")
hotel_df
  

c:\users\etrez\anaconda3\envs\pythondata2\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Lat,Lng,Country,Hotel Name
9,harper,4.3750,-7.7169,LR,Harper
94,alvaraes,-3.2208,-64.8042,BR,Alvarães
116,oistins,13.0667,-59.5333,BB,Bridgetown
119,banjar,-8.1900,114.9675,ID,Banjar
171,bambous virieux,-20.3428,57.7575,MU,Vieux Grand Port
274,the valley,18.2170,-63.0578,AI,The Valley
474,bathsheba,13.2167,-59.5167,BB,Wilson Hill
515,itacare,-14.2775,-38.9967,BR,Itacaré


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
